## 0. Pré requisitos para funcionamento das respectivas partes

### Importações de bibliotecas

In [1]:
from ipywidgets import Dropdown, VBox, Button, Output, Text
from IPython.display import display
from pysus.online_data import IBGE
from pysus import SIH
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geobr
import os

from merradownload.merra_scraping import baixar_merra

## 01 - Baixar shapefiles

In [2]:
municipios_coord_df = pd.read_csv(r"../data/utils/municipios_coord.csv", sep=";")
estados = sorted(municipios_coord_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(municipios_coord_df[municipios_coord_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(options=estados, description='Estado:')
municipios_dd = Dropdown(options=municipios_por_estado[estados[0]], description='Município:')
baixar_btn = Button(description="Baixar e Plotar Shapefile")
output_mapa = Output()

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

def baixar_plotar_shapefile(b):
    uf = estados_dd.value
    municipio = municipios_dd.value

    localizar_municipio = geobr.lookup_muni(name_muni=municipio)
    localizar_municipio = localizar_municipio[localizar_municipio['abbrev_state'] == uf]

    output_mapa.clear_output()
    
    with output_mapa:
        print(f"Baixando shapefile para {municipio}, {uf}...")
        code_muni = localizar_municipio.iloc[0]['code_muni']
        gdf = geobr.read_municipality(code_muni=code_muni)
        print("Download concluído.")

        save_dir = rf"../data/shapefiles/{uf}-{municipio.replace(' ', '_')}/"
        os.makedirs(save_dir, exist_ok=True)
        
        # Salva o shapefile
        filepath = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}.shp")
        gdf.to_file(filepath, driver='ESRI Shapefile')
        print(f"Shapefile salvo em: {filepath}")

        # Salva o CSV com código IBGE
        try:
            ibge_df = pd.read_csv(r"../data/utils/municipios_ibge.csv")
            # Tenta encontrar pelo código IBGE (garantindo que ambos sejam do mesmo tipo, ex: float/int)
            # O geobr retorna float, o csv pode ter int.
            city_ibge_info = ibge_df[ibge_df['codigo_ibge'] == code_muni]
            
            if city_ibge_info.empty:
                # Fallback: Tenta pelo nome do município
                city_ibge_info = ibge_df[ibge_df['Nome_Município'] == municipio]
            
            if not city_ibge_info.empty:
                csv_path = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}_ibge.csv")
                city_ibge_info.to_csv(csv_path, index=False)
                print(f"CSV IBGE salvo em: {csv_path}")
            else:
                print("Aviso: Não foi possível encontrar informações no arquivo municipios_ibge.csv")
        except Exception as e:
            print(f"Erro ao salvar CSV IBGE: {e}")

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        gdf.plot(ax=ax, facecolor='#add8e6', edgecolor='black')
        ax.set_title(f"Mapa de {municipio} - {uf}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.grid(True)
        plt.show()

baixar_btn.on_click(baixar_plotar_shapefile)

display(VBox([estados_dd, municipios_dd, baixar_btn, output_mapa]))